In [1]:
import sddk
import pandas as pd
pd.options.display.max_columns = 1000 # to see all columns
import json
import re
import geopandas as gpd
from shapely.geometry import Point
from functools import partial
from shapely.ops import transform
import pyproj
import warnings
warnings.filterwarnings('ignore')
import numpy as np
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt

In [2]:
EDHCSg = sddk.read_file("EDHCSg.geojson", "gdf", "a9237c5ea642d4714bcdefb03f70a1f4")

reading file located in a public folder


In [3]:
# to train the model, we will work with the EDH subset of the data
EDHg = EDHCSg[EDHCSg["EDH-ID"].notnull()] 

In [9]:
# first we will create prediction-response-pairs as a list of tuples
pred_resp_pairs = [(tup0, tup1) for tup0, tup1 in zip(EDHg["type_of_inscription_clean"].tolist(), EDHg["clean_text_interpretive_word"].tolist())]
pred_resp_pairs[:5]

[('honorific inscription',
  'Fortissimo et piissimo Caesari domino nostro Galerio Valerio Maximiano Pio Felici Invicto Coranius Titianus vir perfectissimus praeses provinciae veteris Epiri numini eorum dicatissimus'),
 ('mile-/leaguestone',
  'Imperatori Caesari Marco Annio Floriano Pio Felici Augusto patri patriae milia passuum III Imperatori Caesari Marco Aurelio Probo Pio Felici Augusto milia passuum II'),
 ('public legal inscription',
  'Tiberius Claudius Caesar Augustus Germanicus pontifex maximus tribunicia potestate VIIII imperator XVI pater patriae dicit cum et colonias et municipia non solum Italiae verum etiam provinciarum item civitatium cuius que provinciae lebare oneribus vehiculorum praebendorum saepe temptavissem et cum satis multa remedia invenisse mihi viderer potuit tamen nequitiae hominum C RAS RVM aut falsa nomina ius dedit turo ante diem'),
 ('votive inscription',
  'Pro salute Imperatorum Caesaris Luci Septimi Severi et Marci Aureli Antonini Auggustorum et Publi 

In [11]:
import nltk

In [21]:
pred_resp_pairs[0][1]

'Fortissimo et piissimo Caesari domino nostro Galerio Valerio Maximiano Pio Felici Invicto Coranius Titianus vir perfectissimus praeses provinciae veteris Epiri numini eorum dicatissimus'

In [22]:
bigrams = list(nltk.bigrams([pred_resp_pairs[0][1].split()]))
bigrams

[]

In [18]:
bigrams

[]